# Use of Machine Learning Methods with Keras for predicting cancer on non-carcinogenic cell line with SNP data

## Objective

I want to predict carcinogenic cell line based on:
    
    - SNP transversion/transition
    - maf score of SNP
    - functional element associated with the SNP
    - cell line

In [3]:
# imports utilites
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import time

In [4]:
df = pd.read_csv('functionalElementDataARBindingProstate.csv')
df.tail()

,RS_ID,chr,pos,ref,alt,scoreA,scoreB,functional_element,n_experiment,file_type,cell_line,cancer_type,cell_line_cancer
457928,rs997633,5,132219304,A,T,0.1052,0.112529,CTCF-human,1,narrow,RWPE1,prostate,normal
457929,rs997633,5,132219304,A,T,0.1052,0.112529,CTCF-human,1,narrow,RWPE2,prostate,normal
457930,rs997633,5,132219304,A,T,0.1052,0.112529,CTCF-human,1,narrow,VCaP,prostate,cancer
457931,rs997633,5,132219304,A,T,0.1052,0.112529,EP300-human,1,narrow,prostate gland,NaN,normal
457932,rs997633,5,132219304,A,T,0.1052,0.112529,POLR2AphosphoS5-human,1,narrow,prostate gland,NaN,normal


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457933 entries, 0 to 457932
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   RS_ID               457933 non-null  object 
 1   chr                 457933 non-null  object 
 2   pos                 457933 non-null  int64  
 3   ref                 457933 non-null  object 
 4   alt                 457933 non-null  object 
 5   scoreA              379835 non-null  float64
 6   scoreB              392889 non-null  float64
 7   functional_element  457933 non-null  object 
 8   n_experiment        457933 non-null  int64  
 9   file_type           457933 non-null  object 
 10  cell_line           457933 non-null  object 
 11  cancer_type         285609 non-null  object 
 12  cell_line_cancer    457933 non-null  object 
dtypes: float64(2), int64(2), object(9)
memory usage: 45.4+ MB


## Data cleaning

In [6]:
df.isnull().sum() # check for null value

RS_ID                      0
chr                        0
pos                        0
ref                        0
alt                        0
scoreA                 78098
scoreB                 65044
functional_element         0
n_experiment               0
file_type                  0
cell_line                  0
cancer_type           172324
cell_line_cancer           0
dtype: int64

### Feature engineering of cancer_type
Because I have 172324 values empty on cancer_type, I don't want to eliminate it, I engineer the feature, changing label ('prostate','empty') with (1,2) labels.

Procedure for feature engineering on cancer_type:
possible values ('empty' or 'prostate')

1) replace blank value with NaN

2) replace Nan value with 2, the numeric label for non-carcinogenic 

3) replace 'prostate' value with 1, the numeric label for prostate

In [7]:
df['cancer_type']=df['cancer_type'].replace(r'^\s*$', np.nan, regex=True)
df['cancer_type']=df['cancer_type'].fillna(1) # NORMAL = 1
df['cancer_type']=df['cancer_type'].replace('prostate',0, regex=True) # PROSTATE = 0

In [8]:
df2=df.copy() # copy the dataframe to new dataframe

In [9]:
df2.head()

,RS_ID,chr,pos,ref,alt,scoreA,scoreB,functional_element,n_experiment,file_type,cell_line,cancer_type,cell_line_cancer
0,rs10201930,2,189874460,G,A,0.0719,0.065940,H3K36me3-human,1,narrow,prostate,2,normal
1,rs10178969,2,190315723,A,C,0.0657,0.068314,EZH2-human,1,narrow,PC-3,1,cancer
2,rs10178969,2,190315723,A,C,0.0657,0.068314,H3K36me3-human,1,narrow,PC-3,1,cancer
3,rs1037532,18,50038602,T,C,0.0280,0.034459,H3K27me3-human,1,narrow,PC-3,1,cancer
4,rs10039204,5,101180077,T,C,NaN,NaN,EZH2-human,1,narrow,PC-3,1,cancer


In [10]:
df2.isnull().sum() # Check if there's null values

RS_ID                     0
chr                       0
pos                       0
ref                       0
alt                       0
scoreA                78098
scoreB                65044
functional_element        0
n_experiment              0
file_type                 0
cell_line                 0
cancer_type               0
cell_line_cancer          0
dtype: int64

### Feature engineering on ScoreA and ScoreB

Create a mean value of ScoreA and ScoreB that I call ScoreC.

In [11]:
st=time.time()
def adjust_score_mark1(df): # SLOWER than mark2
    """ try to use every maf possible
        if each maf score miss, drop the row
        if just one miss, use as scoreC the other
        if both are present use the mean for calculate scoreC
    """
    for index, row in df.iterrows():
        if not row['scoreB']:
            row['scoreC']=row['scoreA']
        elif not row['scoreA']:
            row['scoreC']=row['scoreB']
        elif row['scoreB'] and row['scoreA']:
            row['scoreC']=(row['scoreA']+row['scoreB'])/2
        else:
            pass
    return df
print(' time :',time.time() - st)

 time : 0.0002346038818359375


In [12]:
def adjust_score_mark2(df): # FASTER than mark1
    
    """ try to use every maf possible
            if each maf score miss, drop the row
            if just one miss, use as scoreC the other
            if both are present use the mean for calculate scoreC
    """
    list_=[]
    for row in df.itertuples(index=True, name='Pandas'):
        A,B = getattr(row, "scoreA"), getattr(row, "scoreB")

        if A and B:
            C=(A+B)/2
        if A and not B:
            C = A
        if B and not A:
            C = 0
            
        list_.append(C)
    df['scoreC']=list_

    return df

In [13]:
df3 = adjust_score_mark2(df2) # wait for long time for calculation 

In [14]:
df3.head() # show 

,RS_ID,chr,pos,ref,alt,scoreA,scoreB,functional_element,n_experiment,file_type,cell_line,cancer_type,cell_line_cancer,scoreC
0,rs10201930,2,189874460,G,A,0.0719,0.065940,H3K36me3-human,1,narrow,prostate,2,normal,0.068920
1,rs10178969,2,190315723,A,C,0.0657,0.068314,EZH2-human,1,narrow,PC-3,1,cancer,0.067007
2,rs10178969,2,190315723,A,C,0.0657,0.068314,H3K36me3-human,1,narrow,PC-3,1,cancer,0.067007
3,rs1037532,18,50038602,T,C,0.0280,0.034459,H3K27me3-human,1,narrow,PC-3,1,cancer,0.031230
4,rs10039204,5,101180077,T,C,NaN,NaN,EZH2-human,1,narrow,PC-3,1,cancer,NaN


In [15]:
df3.isnull().sum() # check now for null values

RS_ID                     0
chr                       0
pos                       0
ref                       0
alt                       0
scoreA                78098
scoreB                65044
functional_element        0
n_experiment              0
file_type                 0
cell_line                 0
cancer_type               0
cell_line_cancer          0
scoreC                79635
dtype: int64

Drop values on scoreA and scoreB, and drop null values of scoreC.

Score = **maf score** from TOPmed and from 1000genomes.

In [16]:
df3.columns

Index(['RS_ID', 'chr', 'pos', 'ref', 'alt', 'scoreA', 'scoreB',
       'functional_element', 'n_experiment', 'file_type', 'cell_line',
       'cancer_type', 'cell_line_cancer', 'scoreC'],
      dtype='object')

In [17]:
df3=df3.drop(['scoreA'], axis=1) # drop values
df3=df3.drop(['scoreB'], axis=1) # drop values
df3=df3.drop(df3.loc[df3['scoreC'].isnull()].index) # drop null values

In [18]:
df3.isnull().sum() # Check if there's null values

RS_ID                 0
chr                   0
pos                   0
ref                   0
alt                   0
functional_element    0
n_experiment          0
file_type             0
cell_line             0
cancer_type           0
cell_line_cancer      0
scoreC                0
dtype: int64

In [19]:
df3.head()

,RS_ID,chr,pos,ref,alt,functional_element,n_experiment,file_type,cell_line,cancer_type,cell_line_cancer,scoreC
0,rs10201930,2,189874460,G,A,H3K36me3-human,1,narrow,prostate,2,normal,0.068920
1,rs10178969,2,190315723,A,C,EZH2-human,1,narrow,PC-3,1,cancer,0.067007
2,rs10178969,2,190315723,A,C,H3K36me3-human,1,narrow,PC-3,1,cancer,0.067007
3,rs1037532,18,50038602,T,C,H3K27me3-human,1,narrow,PC-3,1,cancer,0.031230
5,rs1002223,7,21172073,A,C,EZH2-human,1,narrow,PC-3,1,cancer,0.301696


## Feature engineering

In [20]:
df3['file_type'].value_counts()

narrow    378298
Name: file_type, dtype: int64

I eliminate file_type, because 100% of file_type=='narrow', so it's an useless feature. File_type

In [21]:
df3=df3.drop(['file_type'], axis=1)

Number of experiment replicate :

In [22]:
df3['n_experiment'].value_counts()

1    351987
2     26311
Name: n_experiment, dtype: int64

In [23]:
print(' Number of experiment with 1 replicate = ',round(df3['n_experiment'].value_counts()[1]/len(df3),5),'%')
print(' Number of experiment with 2 replicate = ',round(df3['n_experiment'].value_counts()[2]/len(df3),5),'%')

 Number of experiment with 1 replicate =  0.93045 %
 Number of experiment with 2 replicate =  0.06955 %


I drop it, because probably this feature it will be useless to predict prostate cancer due to the majority of 1 replicate. 

In [24]:
df3=df3.drop(['n_experiment'], axis=1)

### Feature engineering chr, pos, RS_ID

From: chr, pos, RS_ID I want to identify SNP with just one of those. I choose to modify ID to integer value to label SNP and drop the other two.

In [25]:
df3=df3.drop(['chr','pos'], axis=1)

In [26]:
df4=df3.copy()

In [27]:
df4.head()

,RS_ID,ref,alt,functional_element,cell_line,cancer_type,cell_line_cancer,scoreC
0,rs10201930,G,A,H3K36me3-human,prostate,2,normal,0.068920
1,rs10178969,A,C,EZH2-human,PC-3,1,cancer,0.067007
2,rs10178969,A,C,H3K36me3-human,PC-3,1,cancer,0.067007
3,rs1037532,T,C,H3K27me3-human,PC-3,1,cancer,0.031230
5,rs1002223,A,C,EZH2-human,PC-3,1,cancer,0.301696


Use RS_ID as categorical value to distinguish different SNP.
Remove 'rs' from all data in RS_ID and transform the colum from char to integer.

In [28]:
df4['RS_ID'] = df4['RS_ID'].str.replace("rs","") # replace rs with nothing
df4.RS_ID = pd.to_numeric(df4.RS_ID, errors='coerce') # convert rs_id to integer

In [29]:
df4.head()

,RS_ID,ref,alt,functional_element,cell_line,cancer_type,cell_line_cancer,scoreC
0,10201930,G,A,H3K36me3-human,prostate,2,normal,0.068920
1,10178969,A,C,EZH2-human,PC-3,1,cancer,0.067007
2,10178969,A,C,H3K36me3-human,PC-3,1,cancer,0.067007
3,1037532,T,C,H3K27me3-human,PC-3,1,cancer,0.031230
5,1002223,A,C,EZH2-human,PC-3,1,cancer,0.301696


In [30]:
len(df4['cell_line'].unique()) # different cell lines

11

Find the number of different Functional Elements:

In [31]:
len(df4['functional_element'].unique()) # different Functional element

20

### Feature engineering cell_line_cancer

In [32]:
df5=df4.copy()

In [33]:
df5.head()

,RS_ID,ref,alt,functional_element,cell_line,cancer_type,cell_line_cancer,scoreC
0,10201930,G,A,H3K36me3-human,prostate,2,normal,0.068920
1,10178969,A,C,EZH2-human,PC-3,1,cancer,0.067007
2,10178969,A,C,H3K36me3-human,PC-3,1,cancer,0.067007
3,1037532,T,C,H3K27me3-human,PC-3,1,cancer,0.031230
5,1002223,A,C,EZH2-human,PC-3,1,cancer,0.301696


On cell_line_cancer substitute cancer with 1 and normal with 2 

In [34]:
df5['cell_line_cancer'] = df5['cell_line_cancer'].str.replace("cancer","0")
df5['cell_line_cancer'] = df5['cell_line_cancer'].str.replace("normal","1")
df5.cell_line_cancer = pd.to_numeric(df5.cell_line_cancer, errors='coerce')

In [35]:
df5.head()

,RS_ID,ref,alt,functional_element,cell_line,cancer_type,cell_line_cancer,scoreC
0,10201930,G,A,H3K36me3-human,prostate,2,2,0.068920
1,10178969,A,C,EZH2-human,PC-3,1,1,0.067007
2,10178969,A,C,H3K36me3-human,PC-3,1,1,0.067007
3,1037532,T,C,H3K27me3-human,PC-3,1,1,0.031230
5,1002223,A,C,EZH2-human,PC-3,1,1,0.301696


### Nucleotide feature engineering

In [36]:
# REFERENCE
df5['ref'] = df5['ref'].str.replace("T","1") # T=1
df5['ref'] = df5['ref'].str.replace("C","2") # C=2
df5['ref'] = df5['ref'].str.replace("A","3") # A=3
df5['ref'] = df5['ref'].str.replace("G","4") # G=4

# ALTERNATIVE (mutated)
df5['alt'] = df5['alt'].str.replace("T","1") 
df5['alt'] = df5['alt'].str.replace("C","2")
df5['alt'] = df5['alt'].str.replace("A","3")
df5['alt'] = df5['alt'].str.replace("G","4")

# transform Nucleotide label into integer
df5.alt = pd.to_numeric(df5.alt, errors='coerce')
df5.ref = pd.to_numeric(df5.ref, errors='coerce')


In [37]:
df5.head()

,RS_ID,ref,alt,functional_element,cell_line,cancer_type,cell_line_cancer,scoreC
0,10201930,4,3,H3K36me3-human,prostate,2,2,0.068920
1,10178969,3,2,EZH2-human,PC-3,1,1,0.067007
2,10178969,3,2,H3K36me3-human,PC-3,1,1,0.067007
3,1037532,1,2,H3K27me3-human,PC-3,1,1,0.031230
5,1002223,3,2,EZH2-human,PC-3,1,1,0.301696


In [38]:
df6=df5.copy()

### Feature engineering cell_line
I want to transform each name of cell_line into an integer.

In [39]:
list_name_cell_line=list(df6['cell_line'].unique() )
print(list_name_cell_line)

['prostate', 'PC-3', 'prostate gland', '22Rv1', 'C4-2B', 'epithelial cell of prostate', 'LNCaP clone FGC', 'VCaP', 'RWPE1', 'LNCAP', 'RWPE2']


In [40]:
df6['cell_line']=df6['cell_line'].replace(to_replace=list_name_cell_line , value=range(len(list_name_cell_line))) 

In [41]:
df6.head()

,RS_ID,ref,alt,functional_element,cell_line,cancer_type,cell_line_cancer,scoreC
0,10201930,4,3,H3K36me3-human,0,2,2,0.068920
1,10178969,3,2,EZH2-human,1,1,1,0.067007
2,10178969,3,2,H3K36me3-human,1,1,1,0.067007
3,1037532,1,2,H3K27me3-human,1,1,1,0.031230
5,1002223,3,2,EZH2-human,1,1,1,0.301696


### Feature engineering functional_element
I want to transform each name of functional_element into an integer.

In [42]:
df7=df6.copy()

In [43]:
list_name_functional_element=list(df7['functional_element'].unique() )
print(list_name_functional_element)

['H3K36me3-human', 'EZH2-human', 'H3K27me3-human', 'POLR2A-human', 'EP300-human', 'CTCF-human', 'H3K27ac-human', 'H3K4me3-human', 'H3K4me1-human', 'H3K4me2-human', 'H3K79me2-human', 'H3K9ac-human', 'H4K20me1-human', 'POLR2AphosphoS5-human', 'ZFX-human', 'H2AFZ-human', 'H3F3A-human', 'H3K9me3-human', 'EZH2phosphoT487-human', 'H3K9me2-human']


In [44]:
df7['functional_element']=df7['functional_element'].replace(to_replace=list_name_functional_element , value=range(len(list_name_functional_element))) 

In [45]:
df7.head()

,RS_ID,ref,alt,functional_element,cell_line,cancer_type,cell_line_cancer,scoreC
0,10201930,4,3,0,0,2,2,0.068920
1,10178969,3,2,1,1,1,1,0.067007
2,10178969,3,2,0,1,1,1,0.067007
3,1037532,1,2,2,1,1,1,0.031230
5,1002223,3,2,1,1,1,1,0.301696


Really I don't know if ref and alt serves to the scope as RS_ID identify both nucleotide, the reference and the mutated one, but I keep it.

I want to predict if a normal cell line with such SNPs mutation is a prostate cancer.
Predict cancer_type with cell_line_cancer = 'normal' (2).
So I drop out all cell_line_cancer==1 (because they are already cancer).

    **Prediction**
label=( not-protate-cancer , prostate-cancer ) 
using just non-cancer cell line


#### Drop 
Drop out all cell_line_cancer==1 (that is cancer)
(I could do it before) 

In [46]:
df7.head()

,RS_ID,ref,alt,functional_element,cell_line,cancer_type,cell_line_cancer,scoreC
0,10201930,4,3,0,0,2,2,0.068920
1,10178969,3,2,1,1,1,1,0.067007
2,10178969,3,2,0,1,1,1,0.067007
3,1037532,1,2,2,1,1,1,0.031230
5,1002223,3,2,1,1,1,1,0.301696


In [47]:
df7 = df7.drop(df7[df7.cell_line_cancer == 1].index)

In [48]:
df7.head()

,RS_ID,ref,alt,functional_element,cell_line,cancer_type,cell_line_cancer,scoreC
0,10201930,4,3,0,0,2,2,0.068920
6,1002223,3,2,3,2,2,2,0.301696
8,1037280,2,1,4,2,2,2,0.240341
9,10032255,1,4,5,2,2,2,0.119516
13,10103239,3,4,6,5,2,2,0.012565


Drop cell_line_cancer column because it's all from 'normal cell line'

In [49]:
df7=df7.drop(['cell_line_cancer'], axis=1) # drop values

In [50]:
df8=df7.copy()

In [51]:
df8.tail()

,RS_ID,ref,alt,functional_element,cell_line,cancer_type,scoreC
457927,997633,3,1,5,2,2,0.108865
457928,997633,3,1,5,8,1,0.108865
457929,997633,3,1,5,10,1,0.108865
457931,997633,3,1,4,2,2,0.108865
457932,997633,3,1,13,2,2,0.108865


In [99]:
df8.isnull().sum() # no null value

0
0
0
0
0
0
0


### Machine learning with TensorFlow.Keras
#### Recurrent Neural Networks (LSTM / RNN)

In [53]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [54]:
print(tf.__version__) # version of TensorFlow

2.0.0


In [55]:
# Machine Learning data model

EPOCHS = 3

BATCH_SIZE = 16

P_train = 0.75 # percenteage of train = (1-P_test)


#### DF length

In [56]:
LEN=len(df8) 
print(len(df8))

186093


#### Select features for Input

In [57]:
df9=df8.copy()

In [94]:
df8.dtypes # check if are all numbers (int or float)

RS_ID                   int64
ref                     int64
alt                     int64
functional_element      int64
cell_line               int64
cancer_type             int64
scoreC                float64
dtype: object

In [97]:
list_of_cell_lines = df8['cell_line'].unique()

array([ 0,  2,  5,  8, 10])

In [81]:
df_data = df9[['RS_ID','ref','alt','scoreC']] # actual model

#### Data features for ML 

In [93]:
#df_data = df9[['RS_ID','ref','alt','scoreC']]             #  model 1 | accuracy ~ 20%
#df_data = df8[['RS_ID','ref','alt','cell_line','scoreC']]  #  model 2 | accuracy ~ 23%
df_data = df8[['RS_ID','ref','alt','functional_element','cell_line','scoreC']]  # model 3 | accuracy ~ 23%

In [83]:
NUM_features =len(df_data.iloc[0])

In [84]:
# select feature for OUTPUT (cancer_type) -> binary type:(1,2)
df_label = df8[['cancer_type']]
# df_label.head()

### Train the model
Training the neural network model requires the following steps:

    - Feed the training data to the model. In this example, the training data is in the 
        train_data and train_labels arrays.
    
    - The model learns to associate data and labels.
    
    - You ask the model to make predictions about a test set—in this example,
          the test_data array.
    
    - Verify that the predictions match the labels from the test_labels array.


### Extrapolate Train and Test data

In [85]:
# Generate TRAIN X
df_train_data = df_data.loc[:round(LEN*P_train)]
x_train = df_train_data.to_numpy()

In [86]:
# Generate TRAIN Y
df_train_label = df_label.loc[:round(LEN*P_train)]
y_train = df_train_label.to_numpy()

In [87]:
# Generate TEST X
df_test_data = df_data.loc[round(1-(LEN*P_train)):]
x_test = df_test_data.to_numpy()

In [88]:
# Generate TEST Y
df_test_label = df_label.loc[round(1-(LEN*P_train)):]
y_test = df_test_label.to_numpy()

### Build the Model

In [89]:
model = Sequential()
model.add(Dense(128, input_dim=NUM_features, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

### Compile the Model

In [90]:
model.compile(loss='binary_crossentropy',
              optimizer='RMSprop',
              metrics=['accuracy'])

### Fit the Model

In [91]:
model.fit(x_train, y_train,
          epochs=EPOCHS,
          batch_size=BATCH_SIZE)

Train on 58058 samples
Epoch 1/3
58058/58058 [==============================] - 20s 344us/sample - loss: -21739516759702240.0000 - accuracy: 0.2374
Epoch 2/3
58058/58058 [==============================] - 18s 304us/sample - loss: -1278966232226837504.0000 - accuracy: 0.2375
Epoch 3/3
58058/58058 [==============================] - 18s 302us/sample - loss: -12988883344983248896.0000 - accuracy: 0.2375


In [92]:
# possible Error : IOPub data rate exceeded.
# Solution below to enter into the terminal 
# jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10
# and restart the .ipynb file
score = model.evaluate(x_test, y_test, batch_size = BATCH_SIZE)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

